In [104]:
import sys  
reload(sys)  
sys.setdefaultencoding('utf-8')

In [105]:
import numpy.linalg as LA
import matplotlib.pyplot as plt
#from my_pickle import my_pickle_load, my_pickle_dump

%matplotlib inline

In [106]:
from nltk.tokenize.stanford_segmenter import StanfordSegmenter

segmenter = StanfordSegmenter(path_to_jar='./stanford-segmenter-2014-08-27/stanford-segmenter-3.4.1.jar', 
                              path_to_slf4j='./stanford-segmenter-2014-08-27/slf4j-api.jar',
                              path_to_sihan_corpora_dict='./stanford-segmenter-2014-08-27/data', 
                              path_to_model='./stanford-segmenter-2014-08-27/data/pku.gz', 
                              path_to_dict='./stanford-segmenter-2014-08-27/data/dict-chris6.ser.gz')

In [107]:
segmenter = StanfordSegmenter(path_to_jar='/Users/nhu2000/desktop/nltk/stanford-segmenter-2014-08-27/stanford-segmenter-3.4.1.jar', 
                              path_to_slf4j='/Users/nhu2000/desktop/nltk/stanford-segmenter-2014-08-27/slf4j-api.jar',
                              path_to_sihan_corpora_dict='/Users/nhu2000/desktop/nltk/stanford-segmenter-2014-08-27/data', 
                              path_to_model='/Users/nhu2000/desktop/nltk/stanford-segmenter-2014-08-27/data/pku.gz', 
                              path_to_dict='/Users/nhu2000/desktop/nltk/stanford-segmenter-2014-08-27/data/dict-chris6.ser.gz')

str=u"我在博客园开了一个博客，我的博客名叫伏草惟存，写了一些自然语言处理的文章。"
result = segmenter.segment(str)

print result

In [48]:
import cPickle as pickle
import pandas as pd
import numpy as np

In [49]:
import string
import math
#import glob
from collections import Counter

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

import nltk
from nltk import tokenize
from nltk.stem.porter import *
from nltk.corpus import stopwords

from gensim import matutils
from gensim.models.ldamodel import LdaModel
from gensim.models.lsimodel import LsiModel

from gensim import corpora, models, similarities
from sklearn.metrics.pairwise import linear_kernel


In [51]:
def my_pickle_dump(fname, data):
    with open(fname, 'w') as f:
        pickle.dump(data, f)

def my_pickle_load(fname):
    with open(fname) as f:
        return pickle.load(f)

In [52]:
df = pd.read_csv('/Users/nhu2000/desktop/huizhou/data/complete/hzwh_huixueyuanpu.csv')

In [108]:
df = pd.read_csv('/Users/nhu2000/desktop/huizhou/data/complete/weixin_huizhou.csv')

In [109]:
df.head()

,dt,source,domain,url,title,summary,image,brief,body,raw_page,from
0,2017-02-10,歙县论坛,http://mp.weixin.qq.com,http://mp.weixin.qq.com/s?__biz=MjM5OTc1NDAxMg...,【徽州】《梦里徽州——新安江风情画卷》,NaN,NaN,［汪观清专访］一生痴绝处，无梦到徽州。 汪观清，安徽歙县人。他创作的《梦里..,\n \n\n ...,"<html><head>\n <meta http-equiv=""Conten...",微信
1,2017-02-08,徽州微旅游,http://mp.weixin.qq.com,http://mp.weixin.qq.com/s?__biz=MzA4NDI0MzMwNA...,古村的梦——古建筑博物馆,NaN,NaN,千年的光阴，在轮回里，阴晴圆缺，一场梦醒时分，恍若隔世，重新遇见你眉目缱绻飘零，..,\n \n\n ...,"<html><head>\n <meta http-equiv=""Conten...",微信
2,2017-01-30,徽州范儿,http://mp.weixin.qq.com,http://mp.weixin.qq.com/s?__biz=MzIyOTQ4MjEyNA...,苞芦松 | 传承四百年的徽州小食,NaN,NaN,编辑小语：在徽州，有一种延续了四百年的休闲食品。金黄的外表，清脆的口感，颇像“膨..,\n \n\n ...,"<html><head>\n <meta http-equiv=""Conten...",微信
3,2017-01-30,徽州大论坛,http://mp.weixin.qq.com,http://mp.weixin.qq.com/s?__biz=MzI3MTAwNTA4NA...,【徽州大家谈】墨客典籍与现代生活的交融地,NaN,NaN,徽州古城坐落在歙县县城徽城的中心，也是千年徽州府治的所在地。在这里，记者们不仅参..,\n \n\n ...,"<html><head>\n <meta http-equiv=""Conten...",微信
4,2017-01-28,徽州大论坛,http://mp.weixin.qq.com,http://mp.weixin.qq.com/s?__biz=MzI3MTAwNTA4NA...,【徽州大家谈】时代呼唤中华商魂——我所知的徽商,NaN,NaN,有大专家在视频里向年青人“说”徽商，他告诉人们，历史上的徽商，只不过是从“晋商”..,\n \n\n ...,"<html><head>\n <meta http-equiv=""Conten...",微信


In [110]:
df.info()

In [111]:
df_no_dup = df.drop_duplicates(['title'])

In [112]:
df_no_dup = df_no_dup.drop_duplicates(['url'])

In [113]:
df_no_dup.info()

df_no_dup1 = df_no_dup.drop_duplicates(['content'])


df_no_dup1.info()

In [114]:
df_no_dup.head()

,dt,source,domain,url,title,summary,image,brief,body,raw_page,from
0,2017-02-10,歙县论坛,http://mp.weixin.qq.com,http://mp.weixin.qq.com/s?__biz=MjM5OTc1NDAxMg...,【徽州】《梦里徽州——新安江风情画卷》,NaN,NaN,［汪观清专访］一生痴绝处，无梦到徽州。 汪观清，安徽歙县人。他创作的《梦里..,\n \n\n ...,"<html><head>\n <meta http-equiv=""Conten...",微信
1,2017-02-08,徽州微旅游,http://mp.weixin.qq.com,http://mp.weixin.qq.com/s?__biz=MzA4NDI0MzMwNA...,古村的梦——古建筑博物馆,NaN,NaN,千年的光阴，在轮回里，阴晴圆缺，一场梦醒时分，恍若隔世，重新遇见你眉目缱绻飘零，..,\n \n\n ...,"<html><head>\n <meta http-equiv=""Conten...",微信
2,2017-01-30,徽州范儿,http://mp.weixin.qq.com,http://mp.weixin.qq.com/s?__biz=MzIyOTQ4MjEyNA...,苞芦松 | 传承四百年的徽州小食,NaN,NaN,编辑小语：在徽州，有一种延续了四百年的休闲食品。金黄的外表，清脆的口感，颇像“膨..,\n \n\n ...,"<html><head>\n <meta http-equiv=""Conten...",微信
3,2017-01-30,徽州大论坛,http://mp.weixin.qq.com,http://mp.weixin.qq.com/s?__biz=MzI3MTAwNTA4NA...,【徽州大家谈】墨客典籍与现代生活的交融地,NaN,NaN,徽州古城坐落在歙县县城徽城的中心，也是千年徽州府治的所在地。在这里，记者们不仅参..,\n \n\n ...,"<html><head>\n <meta http-equiv=""Conten...",微信
4,2017-01-28,徽州大论坛,http://mp.weixin.qq.com,http://mp.weixin.qq.com/s?__biz=MzI3MTAwNTA4NA...,【徽州大家谈】时代呼唤中华商魂——我所知的徽商,NaN,NaN,有大专家在视频里向年青人“说”徽商，他告诉人们，历史上的徽商，只不过是从“晋商”..,\n \n\n ...,"<html><head>\n <meta http-equiv=""Conten...",微信


In [115]:
from bs4 import BeautifulSoup

In [116]:
def get_columns3():
    return ('url', 'segments') 
def get_empty_columns3():
    return ('', '') 

In [117]:
idx = 0
new_df = pd.DataFrame(columns = get_columns3())

In [119]:
for i in range(0, len(df_no_dup)):
    url = df_no_dup.loc[i]['url']
    t1 = df_no_dup.loc[i]['body'].decode('utf-8')
    print i, url
    t0 = segmenter.segment(t1)
    new_df.loc[i] = get_empty_columns3()
    new_df.loc[i]['segments'] = t0
    new_df.loc[i]['url'] = url

In [120]:
new_df.to_csv('/Users/nhu2000/desktop/huizhou/data/complete/weixin_huizhou_segments.csv', encoding='utf-8', index=False)

In [79]:
titles = df_no_dup['title'].str.decode('utf-8')

In [59]:
Baidu_CH_Stopwords_list = ['按, 按照, 俺, 俺们, 阿', 
                      '别, 别人, 别处, 别是, 别的, 别管, 别说, 不, 不仅 ,不但, 不光, 不单, 不只, 不外乎, 不如, 不妨, 不尽, 不尽然, 不得, 不怕, 不惟, 不成, 不拘, 不料, 不是, 不比, 不然, 不特, 不独, 不管, 不至于, 不若, 不论, 不过, 不问, 比方, 比如, 比及, 比, 本身, 本着, 本地, 本人, 本, 巴巴, 巴, 并, 并且, 非彼, 彼时, 彼此, 便于, 把, 边, 鄙人, 罢了, 被, 般的',
                     '此间,此次,此时,此外,此处,此地,此,才,才能,朝,朝着,从,从此,从而,除非,除此之外,除开,除外,除了,除,诚然,诚如,出来,出于,曾,趁着,趁,处在,乘,冲',
                     '等等,等到,等,第,当着,当然,当地,当,多,多么,多少,对,对于,对待,对方,对比,得,得了,打,打从,的,的确,的话,但,但凡,但是,大家,大,地,待,都,到,叮咚',
                     '而言,而是,而已,而外,而后,而况,而且,而,尔尔,尔后,尔,二来',
                     '非独,非特,非徒,非但,否则,反过来说,反过来,反而,反之,分别,凡是,凡',
                     '个,个别,固然,故,故此,故而,果然,果真,各,各个,各位,各种,各自,关于具体地说,归齐,归,根据,管,赶,跟,过,该,给,光是',
                     '或者,或曰,或是,或则,或,何,何以,何况,何处,何时,还要,还有,还是,还,后者,很,换言之,换句话说,好,后,和',
                     '即,即令,即使,即便,即如,即或,即若,继而,继后,继之,既然,既是,既往,既,尽管如此,尽管,尽,就要,就算,就是说,就是了,就是,就,据,据此,接着,经,经过,结果,及,及其,及至,加以,加之,例如,介于,几时,几,截至,极了,简言之,竟而,紧接着,距,较之,较,进而,鉴于,基于,具体说来,兼之,借傥然,今,叫,将',
                     '可,可以,可是,可见,开始,开外,况且,靠,看',
                     '来说,来自,来着,来,两者,临,类如,论,赖以,连,连同,离',
                     '莫若,莫如,莫不然,假使,假如,假若,某,某个,某些,某某,漫说,没奈何,每当,每,慢说,冒',
                     '哪个,哪些,哪儿,哪天,哪年,哪怕,哪样,哪边,哪里,那里,那边,那般,那样,那时,那儿,那会儿,那些,那么样,那么些,那么,那个,那,乃,乃至,乃至于,宁肯,宁愿,宁可,宁,能,能否,你,你们,您,拿,难道说,内,哪',
                     '凭借,凭,旁人,譬如, 譬喻',
                     '且,且不说,且说,其,其一,其中,其二,其他,其余,其它,其次,前后,前此,前者,起见,起,全部,全体,恰恰相反,岂但,却,去',
                     '若非,若果,若是,若夫,若,另,另一方面,另外,另悉,如若,如此,如果,如是,如同,如其,如何,如下,如上所述,如上,如,然则,然后,然而,任,任何,任凭,仍,仍旧,人家,人们,人,让',
                     '甚至于,甚至,甚而,甚或,甚么,甚且,什么,什么样,上,上下,虽说,虽然,虽则,虽,孰知,孰料,始而,所,所以,所在,所幸,所有,是,是以,是的,设使,设或,设若,谁,谁人,谁料,谁知,随着,随时,随后,随,顺着,顺,受到,使得,使,似的,尚且,庶几,庶乎,时候,省得,说来,首先',
                     '倘,倘使,倘或,倘然,倘若,同,同时,他,他人,他们们,她们,她,它们,它,替代,替,通过,腾', 
                     '这里,这边,这般,这次,这样,这时,这就是说,这儿,这会儿,这些,这么点儿,这么样,这么些,这么,这个,这一来,这,正是,正巧,正如,正值',
                     '万一,为,为了,为什么,为何,为止,为此,为着,无论,无宁,无,我们,我,往,望,惟其,唯有',
                     '下,向着,向使,向,先不先,相对而言,许多,像,小,些',
                     '一,一些,一何,一切,一则,一方面,一旦,一来,一样,一般,一转眼,,由此可见,由此,由是,由于,由,用来,因而,因着,因此,因了,因为,因,要是,要么,要不然,要不是,要不,要,与,与其,与其说,与否,与此同时,以,以上,以为,以便,以免,以及,以故,以期,以来,以至,以至于,以致,己,已,已矣,有,有些,有关,有及,有时,有的,沿,沿着,于,于是,于是乎,云云,云尔,依照,依据,依,余外,也罢,也好,也,又及,又,抑或,犹自,犹且,用,越是',
                     '只当,只怕,只是,只有,只消,只要,只限,再,再其次,再则,再有,再者,再者说,再说,自身,自打,自己,自家,自后,自各儿,自从,自个儿,自,怎样,怎奈,怎么样,怎么办,怎么,怎,至若,至今,至于,至,纵然,纵使,纵令,纵,之,之一,之所以,之类,着呢,着,眨眼,总而言之,总的说来,总的来说,总的来看,总之,在于,在下,在,诸,诸位,诸如,咱们,咱,作为,只,最,照着,照,直到,综上所述,贼死,逐步,遵照,遵循,针对,致,者,则甚,则',
                     '咳,哇,哈,哈哈,哉,哎,哎呀,哎哟,哗,哟,哦,哩,矣哉,矣乎,矣,焉,毋宁,欤,嘿嘿,嘿,嘻,嘛,嘘,嘎登,嘎,嗳,嗯,嗬,嗡嗡,嗡,喽,喔唷,喏,喂,啷当,啪达,啦,啥,啐,啊,唉,哼唷,哼,咧,咦,咚,咋,呼哧,呸,呵呵,呵,呢,呜呼,呜,呗,呕,呃,呀,吱,吧哒,吧,吗,吓,兮,儿,亦,了,乎']

In [60]:
def is_not_chinese_symbol(token):
    if token == u'\u201c' or token == u'\u201d':
        return False
    elif token == u'\uff0c' or token == u'\u3002':
        return False
    elif token == u'\u3001' or token == u'\u300b':
        return False
    elif token == u'\u300a' or token == u'\uff1a':
        return False
    elif token == u'\uff1b' or token == u'\uff09':
        return False
    elif token == u')' or token == u'(':
        return False
    elif token == u'\uff08' or token == u'\u4e00':
        return False
    elif token == u'-' or token == u':':
        return False
    elif token == u'\u548c' or  token == u'>': 
        return False
    elif token == u'\xb7' or  token == u'\u2014\u2014':         
        return False


    return True

In [61]:
combined_str = ''.decode('utf-8')
for words in Baidu_CH_Stopwords_list:
    combined_str = combined_str + words.decode('utf-8') 

print combined_str

In [62]:
Baidu_CH_Stopwords = combined_str.split(',')

In [67]:
def get_columns2():
    return ('token title', 'flag') 
def get_empty_columns2():
    return ('', '') 

print len(get_columns()), len(get_empty_columns())

In [ ]:
for i in range(0, len(df_final)):
    if(i < 10):
        print 'row ----', i
        new_df.loc[i]= get_empty_columns()
        t0 = df_final.loc[i]['title'].decode('utf-8')
        t1 = segmenter.segment(t0)
        t2 = t1.split(' ')
        tokens_no_stopwords = [token for token in t2 if token not in Baidu_CH_Stopwords and is_not_chinese_symbol(token)] 
        new_df.loc[i]['t_tokens'] = tokens_no_stopwords   

In [64]:
idx = 0
for title in titles:
    print idx, title
    idx += 1

In [68]:
new_df = pd.DataFrame(columns = get_columns2())

In [80]:
idx = 0
for title in titles:
    print 'row ----', idx
    new_df.loc[idx] = get_empty_columns2()
    t0 = title.decode('utf-8')
    t1 = segmenter.segment(t0)
    print t1
    new_df.loc[idx]['token title'] = t1
    new_df.loc[idx]['flag'] = 'pos'
    idx += 1

In [82]:
print idx

In [83]:
new_df.head()

,token title,flag
0,【 徽州 】 《 梦里 徽州 —— 新安江 风情 画卷 》\n,pos
1,古 村 的 梦 —— 古建筑 博物馆\n,pos
2,苞芦松 | 传承 四百 年 的 徽州 小食\n,pos
3,【 徽州 大家 谈 】 墨客 典籍 与 现代 生活 的 交融 地\n,pos
4,【 徽州 大家 谈 】 时代 呼唤 中华 商 魂 —— 我 所 知 的 徽商\n,pos


new_df.to_csv('../data/huizhou-hzwh.pos', encoding='utf-8', index=False)

In [84]:
new_df.to_csv('../data/huizhou-weixin.pos', encoding='utf-8', index=False)